# Assignment 7

## Submit as an HTML file

### Print your name below

In [33]:
print("Lynne Zheng")

Lynne Zheng


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [3]:
# Write your answer here:

import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [6]:
results = pd.read_csv("data/results.csv")
results.head(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [18]:
# Write your answer here
print(results.dtypes)
print("number, position, time, milliseconds, fastestLap, rank, fastestLapTime, and fastestLapSpeed need to be converted")

list_replaced = ['\\N']
list_new = [np.nan]

results["race_time_ms"] = results["milliseconds"].replace(list_replaced, list_new)
results["race_time_ms"] = pd.to_numeric(results["race_time_ms"])



resultId              int64
raceId                int64
driverId              int64
constructorId         int64
number               object
grid                  int64
position             object
positionText         object
positionOrder         int64
points              float64
laps                  int64
time                 object
milliseconds         object
fastestLap           object
rank                 object
fastestLapTime       object
fastestLapSpeed      object
statusId              int64
race_time_ms         object
finish_category    category
dtype: object
number, position, time, milliseconds, fastestLap, rank, fastestLapTime, and fastestLapSpeed need to be converted


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [16]:
# Write your answer here

bins = [0, 3, 10, 20, float('inf')]
labels = ["Podium", "Points", "Midfield", "Backmarker"]
results["finish_category"] = pd.cut(results["positionOrder"],
                                    bins = bins,
                                    right = True,
                                    labels = labels)



### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [24]:
# Write your answer here

results["race_duration_minutes"] = results["race_time_ms"] / (1000 * 60)
average_duration = results.groupby("constructorId")["race_duration_minutes"].mean()
top5 = average_duration.sort_values().head(5)
print(top5)


constructorId
35    76.710777
29    77.604125
41    87.046767
16    89.428828
53    89.658852
Name: race_duration_minutes, dtype: float64


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [27]:
# Write your answer here

results["position"] = results["position"].replace(list_replaced, list_new)
results["position"] = pd.to_numeric(results["position"])

drivers_info = results.groupby("driverId").agg(average_finish_position = ("position", "mean"),
                                               total_points = ("points", "sum"),
                                               number_of_races = ("raceId", "count"),
                                               best_finishing_position = ("position", "min"))
drivers_info = drivers_info.sort_values(by = "total_points", ascending = False)
drivers_info.head(10)

,average_finish_position,total_points,number_of_races,best_finishing_position
driverId,,,,
1,3.383803,4396.5,310,1.0
20,5.282443,3098.0,300,1.0
4,6.257732,2061.0,358,1.0
830,4.022388,1983.5,163,1.0
8,6.238596,1873.0,352,1.0
822,6.296089,1778.0,201,1.0
3,6.355932,1594.5,206,1.0
30,3.701245,1566.0,308,1.0
817,8.261538,1307.0,232,1.0


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [32]:
# Write your answer here
results["points_clean"] = results["points"].replace(list_replaced, list_new)
results["grid_clean"] = results["grid"].replace(list_replaced, list_new)
results["laps_clean"] = results["laps"].replace(list_replaced, list_new)
results_clean = results.dropna()

model = sm.ols("points_clean ~ grid_clean + laps_clean", data = results_clean).fit()
print(model.summary())
predicted = pd.DataFrame({"grid_clean" : [3], "laps_clean" : [55]})
predicted_points = model.predict(predicted)
print(predicted_points)


                            OLS Regression Results                            
Dep. Variable:           points_clean   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     1070.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        21:03:30   Log-Likelihood:                -21701.
No. Observations:                7083   AIC:                         4.341e+04
Df Residuals:                    7080   BIC:                         4.343e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.1445      0.192     52.702      0.0